In [1]:
import pandas as pd 
import numpy as np  

from sklearn.model_selection import train_test_split 
import tensorflow as tf 
from tensorflow import keras
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/Churn_Modelling_Cleaned.csv')
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,1,1,1,0.506735,1,False,False
1,0.516,0,0.310811,0.1,0.334031,1,0,1,0.562709,0,False,True
2,0.304,0,0.324324,0.8,0.636357,3,1,0,0.569654,1,False,False
3,0.698,0,0.283784,0.1,0.000000,2,0,0,0.469120,0,False,False
4,1.000,0,0.337838,0.2,0.500246,1,1,1,0.395400,0,False,True


In [3]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(20, input_shape=(11,), activation='relu'), # "more hidden layers can be added" 
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', 
              loss=loss, 
              metrics=['accuracy']) 
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)

    else: 
        model.fit(X_train, y_train, epochs=100, class_weights=weights)

    print("Model evaluation: \n", model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification report: \n", classification_report(y_test, y_preds))

    return y_preds

In [4]:
count_class_0, count_class_1 = df['Exited'].value_counts()

print('Count of class 0: ', count_class_0)
print('Count of class 1: ', count_class_1)

Count of class 0:  7963
Count of class 1:  2037


From above we can see that the customers who exited (class 0) has 7963 records whereas that of class 1 (didn't exit) is 2037. This is a significant difference creating biasness in the dataset.

### Use of Ensemble with under sampling

In [5]:
X = df.drop('Exited', axis = 1) 
y = df['Exited']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y) #"stratify=y" ensures that the class distribution in y_train and y_test is the same as in y

In [7]:
y_train.value_counts()

Exited
0    6370
1    1630
Name: count, dtype: int64

In [8]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [9]:
df2_class_0 = df2[df2['Exited'] == 0]
df2_class_1 = df2[df2['Exited'] == 1]

In [10]:
df2_class_0.shape, df2_class_1.shape

((6370, 12), (1630, 12))

df2_class_0 is the dataframe with majority class

df2_class_1 is the dataframe with minority class

In [11]:
# This function creates a training batch by combining the dataframe of the majority class with dataframe of the minority class
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train['Exited']

    return X_train, y_train

model 1:

In [12]:
X_train, y_train = get_train_batch(df2_class_0, df2_class_1, 0, 2124)

In [13]:
X_train.shape, y_train.shape

((3754, 11), (3754,))

In [14]:
y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6155 - loss: 0.6626
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6408 - loss: 0.6365
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6564 - loss: 0.6243
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6647 - loss: 0.6193
Epoch 5/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6725 - loss: 0.6039
Epoch 6/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6788 - loss: 0.6011
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6892 - loss: 0.5882
Epoch 8/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6996 - loss: 0.5829
Epoch 9/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7032 - loss: 0.5817
Epoch 10/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6912 - loss: 0.5813
Epoch 11/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7170 - loss: 0.5640
Epoch 12/100
118/118 ━━━━━━━━━━━━━━━━━━━━

model 2:

In [15]:
X_train, y_train = get_train_batch(df2_class_0, df2_class_1, 2124, 4248)
y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5468 - loss: 0.7166
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5984 - loss: 0.6658
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6452 - loss: 0.6342
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6637 - loss: 0.6224
Epoch 5/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6895 - loss: 0.6042
Epoch 6/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6735 - loss: 0.6055
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6956 - loss: 0.5880
Epoch 8/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6997 - loss: 0.5852
Epoch 9/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7074 - loss: 0.5702
Epoch 10/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7091 - loss: 0.5743
Epoch 11/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7159 - loss: 0.5576
Epoch 12/100
118/118 ━━━━━━━━━━━━━━━━━━━━

model 3:

In [16]:
X_train, y_train = get_train_batch(df2_class_0, df2_class_1, 4248, 6372)
y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5369 - loss: 0.6846
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6256 - loss: 0.6463
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6311 - loss: 0.6372
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6473 - loss: 0.6253
Epoch 5/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6873 - loss: 0.6057
Epoch 6/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6906 - loss: 0.5950
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6883 - loss: 0.5875
Epoch 8/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6935 - loss: 0.5932
Epoch 9/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7064 - loss: 0.5798
Epoch 10/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7092 - loss: 0.5703
Epoch 11/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7022 - loss: 0.5663
Epoch 12/100
118/118 ━━━━━━━━━━━━━━━━━━━━

In [17]:
print(len(y_pred1))
print(len(y_pred2))
print(len(y_pred3))

2000
2000
2000


In [18]:
y_pred_final = y_pred1.copy() 

for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]

    if n_ones > 1:
        y_pred_final[i] = 1

    else:
        y_pred_final[i] = 0

In [19]:
print('Classification report: \n', classification_report(y_test, y_pred_final))

Classification report: 
               precision    recall  f1-score   support

           0       0.91      0.82      0.86      1593
           1       0.49      0.70      0.58       407

    accuracy                           0.79      2000
   macro avg       0.70      0.76      0.72      2000
weighted avg       0.83      0.79      0.80      2000



Low recall for class 1 (70%) – The model identifies 70% of the minority class (class 1) but still misses a significant number of instances, leading to false negatives.

Class imbalance – The dataset is imbalanced with 1593 instances of class 0 and 407 instances of class 1, causing the model to favor class 0.

Precision-recall tradeoff – The model favors precision for class 0 (91%) and recall for class 1 (70%), leading to a low precision (49%) for class 1. The F1-score for class 1 is low (0.58), indicating poor performance in correctly identifying class 1 instances.